In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs


tf = pd.read_csv('/home/santi/fiuba-tf-tp1-final-set/events.csv',dtype=object)

# Breve descripcion del set de datos.

# Columnas
tf.columns

tf.head()

tf.describe()

tf.info()

# Analizo eventos y relacion de estos con otras variables

# Hay eventos nulos en la columna event?
tf['event'].isnull().sum()

# Comprobamos si hay alguna fecha nula. 
tf['timestamp'].isnull().any()

# Agregamos nuevas columnas para trabajar mejor con el tiempo. (año, mes, dia, dia del año)
tf['timestamp_datetime'] = pd.to_datetime(tf['timestamp'])
tf['timestamp_year'] = tf['timestamp_datetime'].dt.year
tf['timestamp_month'] = tf['timestamp_datetime'].dt.month
tf['timestamp_day'] = tf['timestamp_datetime'].dt.day
tf['timestamp_weekday'] = tf['timestamp_datetime'].dt.weekday_name
tf['timestamp_dayofyear'] = tf['timestamp_datetime'].dt.dayofyear

## Eventos en general

# Nombres de los eventos.

tf['event'].value_counts()

# Ocurrencias de eventos (grafico)

g1 = sns.barplot(x=tf['event'].value_counts().values, y=tf['event'].value_counts().index, orient='h')
g1.set_title("Ocurrencias de eventos", fontsize=15)
g1.set_xlabel("Ocurrencias", fontsize=12)
g1.set_ylabel("Eventos", fontsize=12)

# Analizamos la evolucion de las visitas en la pagina. hacer
#tf.loc(tf[''])

#tf.groupby('timestamp_dayofyear').agg({'person':'d'})

# Analizo el evento viewed product y brand listing.

# Modelo de celular mas visitado.
# Me quedo solo con el evento viewed product.
viewed_product = tf.loc[tf['event'] == 'viewed product']

# Veo si hay algun valor nulo en la columna model.
viewed_product['model'].isnull().sum()

## Celulares mas visitados

# Modelo de celular mas visitado.
viewed_product['model'].value_counts()

# 15 celulares mas visitados.
viewed_product['model'].value_counts().head(15)

# Grafico de barras de los 15 celulares mas visitados.

g2 = sns.barplot(x=viewed_product['model'].value_counts().head(15).values, y=viewed_product['model'].value_counts().head(15).index,palette="Blues_d", orient='h')

g2.set_title("15 celulares mas visitados.", fontsize=15)
g2.set_xlabel("Cantidad de visitas", fontsize=12)
g2.set_ylabel("Celulares", fontsize=12)

# Columnas que puedo relacionar con las visitas a los productos (no tienen valores nulos).
viewed_product.dropna(axis = 'columns').columns

# Visitas de cada celular para cada color.
visitas_por_color = viewed_product.groupby(['model','color']).agg({'timestamp':'count'})
visitas_por_color.columns = ['visitas']
visitas_por_color

# Visitas de cada celular para cada capacidad de almacenamiento.
visitas_por_storage = viewed_product.groupby(['model','storage']).agg({'timestamp':'count'})
visitas_por_storage.columns = ['visitas']
visitas_por_storage

# Visitas de cada celular para cada condicion de producto.
visitas_por_condition = viewed_product.groupby(['model','condition']).agg({'timestamp':'count'})
visitas_por_condition.columns = ['visitas']
visitas_por_condition

cel_mas_visitados = ['iPhone 6','iPhone 5s','iPhone 6S','iPhone 7', 'Samsung Galaxy S7 Edge']
cel_mas_visitados_por_storage = viewed_product[['model','storage']].loc[viewed_product['model'].isin(cel_mas_visitados)]
cel_mas_visitados_por_storage = pd.crosstab(cel_mas_visitados_por_storage.model,cel_mas_visitados_por_storage.storage)
cel_mas_visitados_por_storage

r = [0,1,2,3,4]
cmvps = cel_mas_visitados_por_storage

totals = [i+j+k+l+m for i,j,k,l,m in zip(cmvps['128GB'], cmvps['16GB'],\
                                         cmvps['256GB'], cmvps['32GB'], cmvps['64GB'])]
celcon128 = [i / j * 100 for i,j in zip(cmvps['128GB'], totals)]
celcon16 = [i / j * 100 for i,j in zip(cmvps['16GB'], totals)]
celcon256 = [i / j * 100 for i,j in zip(cmvps['256GB'], totals)]
celcon32 = [i / j * 100 for i,j in zip(cmvps['32GB'], totals)]
celcon64 = [i / j * 100 for i,j in zip(cmvps['64GB'], totals)]

barWidth = 0.85
names = ('S7 Edge','iPhone 5s','iPhone 6','iPhone 6S','iPhone 7')

plt.figure(figsize=(5,3))

plt.bar(r, celcon256, color='#A20101', edgecolor='white', width=barWidth, label = '256GB')

plt.bar(r, celcon128, bottom=celcon256, color='#F44E54', edgecolor='white', width=barWidth, label = '128GB')

plt.bar(r, celcon64, bottom=[i+j for i,j in zip(celcon256, celcon128)], color='#FDDB5E', edgecolor='white'\
                                                , width=barWidth, label = '64GB')

plt.bar(r, celcon32, bottom=[i+j+k for i,j,k in zip(celcon256, celcon128,celcon64)], color='#BAF1A1'\
                                                    , edgecolor='white', width=barWidth, label = '32GB')

plt.bar(r, celcon16, bottom=[i+j+k+l for i,j,k,l in zip(celcon256, celcon128,celcon64,celcon32)], color='#76AD3B'\
                                                        , edgecolor='white', width=barWidth, label = '16GB')

plt.xticks(r, names,fontsize=9)
plt.title('5 celulares mas visitados y su almacenamiento')
plt.xlabel('celulares')
plt.ylabel('porcentaje')

plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
 
plt.show()

cel_mas_visitados_por_color = viewed_product[['model','color']].loc[viewed_product['model'].isin(['iPhone 5s','iPhone 6','iPhone 6S','iPhone 7','iPhone 7 Plus'])]
cel_mas_visitados_por_color = pd.crosstab(cel_mas_visitados_por_color.model,cel_mas_visitados_por_color.color)
cel_mas_visitados_por_color

r = [0,1,2,3,4]
cmvpc = cel_mas_visitados_por_color

totals = [i+j+k+l+m+n+o for i,j,k,l,m,n,o in zip(cmvpc['Cinza espacial']\
                                                 , cmvpc['Dourado'],cmvpc['Ouro Rosa'], cmvpc['Prateado']\
                                                 , cmvpc['Preto Brilhante'],cmvpc['Preto Matte'],cmvpc['Vermelho'])]

celconCinza = [i / j * 100 for i,j in zip(cmvpc['Cinza espacial'], totals)]
celconDourado = [i / j * 100 for i,j in zip(cmvpc['Dourado'], totals)]
celconRosa = [i / j * 100 for i,j in zip(cmvpc['Ouro Rosa'], totals)]
celconPrateado = [i / j * 100 for i,j in zip(cmvpc['Prateado'], totals)]
celconBrilhante = [i / j * 100 for i,j in zip(cmvpc['Preto Brilhante'], totals)]
celconMatte = [i / j * 100 for i,j in zip(cmvpc['Preto Matte'], totals)]
celconVermelho = [i / j * 100 for i,j in zip(cmvpc['Vermelho'], totals)]

barWidth = 0.5
names = ('iPhone 5s','iPhone 6','iPhone 6S','iPhone 7','iPhone 7 Plus')

plt.figure(figsize=(5,3))

plt.bar(r, celconCinza, color='gray', edgecolor='white', width=barWidth, label = 'Gris')

plt.bar(r, celconDourado, bottom=celconCinza, color='gold', edgecolor='white', width=barWidth, label = 'Dorado')

plt.bar(r, celconRosa, bottom=[i+j for i,j in zip(celconCinza, celconDourado)], color='pink', edgecolor='white', width=barWidth, label = 'Rosa')

plt.bar(r, celconPrateado, bottom=[i+j+k for i,j,k in zip(celconCinza, celconDourado,celconRosa)], color='silver', edgecolor='white', width=barWidth, label = 'Plateado')

plt.bar(r, celconBrilhante, bottom=[i+j+k+l for i,j,k,l in zip(celconCinza, celconDourado,celconRosa,celconPrateado)], color='black', edgecolor='white', width=barWidth, label = 'Negro brillante')

plt.bar(r, celconMatte, bottom=[i+j+k+l+m for i,j,k,l,m in zip(celconCinza, celconDourado,celconRosa,celconPrateado,celconBrilhante)], color='#181609', edgecolor='white', width=barWidth, label = 'Negro matte')

plt.bar(r, celconVermelho, bottom=[i+j+k+l+m+n for i,j,k,l,m,n in zip(celconCinza, celconDourado,celconRosa,celconPrateado,celconBrilhante,celconMatte)], color='#b61609', edgecolor='white', width=barWidth, label = 'Rojo')




plt.xticks(r, names,fontsize=9)
plt.title('5 iPhones mas visitados y su color')
plt.xlabel('celulares')
plt.ylabel('porcentaje')

plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
 
plt.show()


## Visitas a un producto en el tiempo

# Vemos las visitas a un producto segun dia de la semana y mes.
visitas_prod_mes_dia_semana = viewed_product.groupby(['timestamp_month','timestamp_weekday']).agg({'event':'count'})\
                .pivot_table(index = 'timestamp_weekday',columns = 'timestamp_month',values = 'event')
visitas_prod_mes_dia_semana

visitas_prod_mes_dia_semana = sns.heatmap(visitas_prod_mes_dia_semana,  cmap="YlGnBu")
visitas_prod_mes_dia_semana.set_title("Visitas a un producto segun mes y dia de la semana", fontsize=22)
visitas_prod_mes_dia_semana.set_xlabel("Mes de la visita",fontsize=18)
visitas_prod_mes_dia_semana.set_ylabel("Dia de la visita", fontsize=18)

# Vemos las visitas a un producto segun mes y dia del mes.
visitas_prod_mes_dia = viewed_product.groupby(['timestamp_month','timestamp_day']).agg({'event':'count'})\
                .pivot_table(index = 'timestamp_month',columns = 'timestamp_day',values = 'event')
visitas_prod_mes_dia

visitas_prod_mes_dia = sns.heatmap(visitas_prod_mes_dia,  cmap="YlGnBu")
visitas_prod_mes_dia.set_title("Visitas a un producto segun mes y dia del mes", fontsize=22)
visitas_prod_mes_dia.set_xlabel("Dia del mes de la visita",fontsize=18)
visitas_prod_mes_dia.set_ylabel("Mes de la visita", fontsize=18)

# Visitas a productos segun dia del año
viewed_product.groupby('timestamp_dayofyear').count().loc[:,'event'].plot(linewidth=2,figsize=(8,4))


# Veo con que puedo relacionar cada evento.
hay otras columnas que pueden tener algunos elementos null pero pueden servir igual (son pocos).

rel_event = tf.groupby('event').agg(lambda x: x.isnull().sum()).T
rel_event

## ad campaign hit

“ad campaign hit”: El usuario ingresa al sitio mediante una campana de marketing online.

Me puede interesar analizar url, person , campaign source y el tiempo.

ad_campaign_hit = tf.loc[tf['event'] == 'ad campaign hit']
ad_campaign_hit['campaign_source'].isnull().sum()

# La columna cuenta con 31 valores nulos. Son muy pocos comparados con los datos que tenemos, asi que trabajo
# con el resto.

# Campañas publicitarias mas populares.
ad_campaign_hit['campaign_source'].value_counts()

# Grafico de popularidad de campañas publicitarias.

g2 = sns.barplot(x=ad_campaign_hit['campaign_source'].value_counts().head(10).values, y=ad_campaign_hit['campaign_source'].value_counts().head(10).index,palette="Blues_d", orient='h')

g2.set_title("10 Campañas mas populares.", fontsize=15)
g2.set_xlabel("Cantidad de visitas", fontsize=12)
g2.set_ylabel("Campañas", fontsize=12)

# Campañas publicitarias mas populares segun el mes.
pd.crosstab(ad_campaign_hit.timestamp_month,ad_campaign_hit.campaign_source)

# Un grafico de lo mismo.
camp_ppales = ['google','criteo', 'rtbhouse', 'zanox', 'emblue']
g = sns.countplot(x="timestamp_month", hue="campaign_source"\
                , tf=ad_campaign_hit.loc[ad_campaign_hit['campaign_source'].isin(camp_ppales)], palette="hls")
g.set_title("Clicks en campañas publicitarias (top 5)", fontsize=18)
g.set_xlabel("Mes", fontsize=18)
g.set_ylabel("Cantidad de clicks", fontsize=18)

# Clicks por dia de las 5 campañas publicitarias mas populares (con mas clicks).
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'google']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,figsize=(8,4),label = 'Google')
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'zanox']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,figsize=(8,4), label = 'Zanox')
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'criteo']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,figsize=(8,4), label = 'Criteo')
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'rtbhouse']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,figsize=(8,4), label = 'Rtbhouse')
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'emblue']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,figsize=(8,4), label = 'Emblue')

plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
plt.ylabel('Cantidad de clicks')


# Ademas podemos ver si existe algun tipo de relacion entre los clicks en la campañas publicitarias
# y las visitas y compras de los productos.

(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'google']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,figsize=(10,4),label = 'Google',alpha = 0.3)
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'zanox']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2, label = 'Zanox',alpha = 0.3)
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'criteo']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2, label = 'Criteo',alpha = 0.3)
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'rtbhouse']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2, label = 'Rtbhouse',alpha = 0.3)
(ad_campaign_hit.loc[ad_campaign_hit['campaign_source'] == 'emblue']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2, label = 'Emblue',alpha = 0.3)
viewed_product.groupby('timestamp_dayofyear').count().loc[:,'event'].plot(logy = True, linewidth=3\
                                                                          ,label = 'Visitas a productos')
tf.loc[tf['event'] == 'conversion'].groupby('timestamp_dayofyear').count().loc[:,'event']\
                                                .plot(linewidth=3,label = 'Conversion de productos')

plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
plt.title('Relacion de la cantidad de clicks en campañas con visitas y compras de productos')


Podemos ver que a medida que aumentan los clicks en las campañas publicitarias, tambien
lo hacen las visitas a productos, lo cual es logico. Algo quizas no tan esperado es que aunque aumentan
las visitas y los clicks en las publicidades, las conversiones no parecen aumentar. En la primer mitad podemos
ver que era mas baja la cantidad de conversiones, pero luego distingimos que estas se mantienen aroximadamente 
constantes a pesar del incremento en las visitas.


ad_campaign_hit['url'].value_counts(normalize = False).drop(labels = '/')
# Podemos ver que direcciones, y por lo tanto que celulares, son las que mas son clickeadas.

ad_campaign_hit['person'].value_counts() # Ver si lo puedo relacionar con algo.

## generic listing

El usuario visita la homepage.

# Podria investigar que paso con los skus, person y el tiempo.

## searched products
El usuario realiza una búsqueda de productos en la interfaz de búsqueda del site.


# Que suelen buscar los usuarios.
searched_products = tf.loc[tf['event'] == 'searched products']
searched_products['search_term'].str.lower().value_counts(normalize = True)

# Podemos ver que la mayoria de las busquedas son de celulares iPhone, seguido por los samsung.

# calculo aproximado.
iphone = searched_products['search_term'].str.lower().str.contains('iphone|5s|5c|6s|7s').sum()
total = searched_products['search_term'].count()
(iphone / total) * 100 
# Los iphone constituyen aproximadamente el 40 %  de las busquedas.

samsung = searched_products['search_term'].str.lower().str.contains('samsung|s3|s4|s5|s6|s7|s8|j7|j5|j3|note|a3|a5|a7').sum()
(samsung / total) * 100
# Los samsung constituyen aproximadamente un 30 % de la busquedas.

motorola = searched_products['search_term'].str.lower().str.contains('moto').sum()
(motorola / total) * 100
# Los motorola constituyen aproximadamente un 10 % de la busquedas.

## search engine hit
“search engine hit”: El usuario ingresa al sitio mediante un motor de búsqueda web.


search_engine_hit = tf.loc[tf['event'] == 'search engine hit']

# Las visitas se dan por un motor de busqueda. Cuales son los mas populares?
search_engine_hit['search_engine'].value_counts()

# Ingresos a traves de un motor de busqueda.

g = sns.barplot(x=search_engine_hit['search_engine'].value_counts().values, y=search_engine_hit['search_engine'].value_counts().index,palette="Blues_d", orient='h')
g.set_xscale('log')
g.set_title("Ingreso a traves de motor de busqueda", fontsize=15)
g.set_xlabel("Cantidad de visitas (escala logaritmica)", fontsize=12)
g.set_ylabel("Motores de busqueda", fontsize=12)

# Ingresos a traves de un motor de busqueda en el tiempo.

(search_engine_hit.loc[search_engine_hit['search_engine'] == 'Google']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,figsize=(10,4),label = 'Google', logy = True)
(search_engine_hit.loc[search_engine_hit['search_engine'] == 'Bing']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,label = 'Bing')
(search_engine_hit.loc[search_engine_hit['search_engine'] == 'Yahoo']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,label = 'Yahoo')
(search_engine_hit.loc[search_engine_hit['search_engine'] == 'Ask']).groupby('timestamp_dayofyear')\
                .count().loc[:,'event'].plot(linewidth=2,label = 'Ask')

plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
plt.title('Ingresos a traves de un motor de busqueda en el tiempo')


## checked out



## visited site
En ppio puedo relacionarlo con person, news vs returning, city, region, country, device_type, screen_resolution, operating_system_version, browser_version y el tiempo

visited_site = tf.loc[tf['event'] == 'visited site']

# Analizo desde donde se hacen las visitas.

# Visitas por pais
visited_site['country'].value_counts()

# Analizo que pasa con el pais Unknown.

visited_site.loc[visited_site['country'] == 'Unknown']['city']

visited_site.loc[visited_site['country'] == 'Unknown']['region']

# No hay informacion tampoco de la ciudad ni de la region.
# Lo saco para poder analizar lo demas.
visited_site_geo = visited_site.loc[(visited_site['country'] != 'Unknown') & (visited_site['city'] != 'Unknown') & \
                                   (visited_site['region'] != 'Unknown')]

# Visitas por pais.
visited_site_geo['country'].value_counts()
# Practicamente todas las visitas fueron de Brazil.

# Visitas por ciudad.
visited_site_geo['city'].value_counts()

# Visitas por region.
visited_site_geo['city'].value_counts()

# Visitas por estado.

# Saco los acentos y pongo todo en minuscula.

estados_br = visited_site_geo['region'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
estados_br = estados_br.str.lower()
estados_br

estados_br.value_counts()

# Me quedo solo con los estados de Brasil.
## GRAFICO PENDIENTE!!!! 
lista_estados_br = ['acre', 'alagoas', 'amapa', 'la amazonas', 'bahia', 'ceara', 'espiritu santo', 'goias', 'maranhao', 'mato grosso', 'mato grosso do sul', 'minas gerais', 'para', 'paraiba', 'parana', 'pernambuco', 'piaui', 'rio de janeiro', 'rio grande do norte', 'rio grande do sul', 'rondonia', 'roraima', 'santa catarina', 'sao paulo', 'sergipe', 'tocantins']
estados_br = estados_br.loc[estados_br.isin(lista_estados_br)]
estados_br.value_counts()



SyntaxError: invalid syntax (<ipython-input-1-9e98e89e0626>, line 219)